In [1]:
from tqdm import tqdm
from multiprocessing import Pool
from typing import Callable, TypeVar
T = TypeVar("T")
def async_multiprocess_loop(func: Callable[[tuple], T], array: list, callback: Callable[[tuple], None]) -> None:
  pbar = tqdm(total=len(array))
  def update(args):
    callback(args)
    pbar.update()
  pool = Pool()
  for item in array:
      pool.apply_async(func, args=(item,), callback=update)
  pool.close()
  pool.join()
  pbar.close()

In [ ]:
import numpy as np
type_offensive_effectivness = {
  #            NOR FIR WAT ELE GRA ICE FIG POI GRO FLY PSY BUG ROC GHO DRA DAR STE FAI
  "normal":   [1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1/2,0  ,1  ,1  ,1/2,1  ],
  "fire":     [1  ,1/2,1/2,1  ,2  ,2  ,1  ,1  ,1  ,1  ,1  ,2  ,1/2,1  ,1/2,1  ,2  ,1  ],
  "water":    [1  ,2  ,1/2,1  ,1/2,1  ,1  ,1  ,2  ,1  ,1  ,1  ,2  ,1  ,1/2,1  ,1  ,1  ],
  "electric": [1  ,1  ,2  ,1/2,1/2,1  ,1  ,1  ,0  ,2  ,1  ,1  ,1  ,1  ,1/2,1  ,1  ,1  ],
  "grass":    [1  ,1/2,2  ,1  ,1/2,1  ,1  ,1/2,2  ,1/2,1  ,1/2,2  ,1  ,1/2,1  ,1/2,1  ],
  "ice":      [1  ,1/2,1/2,1  ,2  ,1/2,1  ,1  ,2  ,2  ,1  ,1  ,1  ,1  ,2  ,1  ,1/2,1  ],
  "fighting": [2  ,1  ,1  ,1  ,1  ,2  ,1  ,1/2,1  ,1/2,1/2,1/2,2  ,0  ,1  ,2  ,2  ,1/2],
  "poison":   [1  ,1  ,1  ,1  ,2  ,1  ,1  ,1/2,1/2,1  ,1  ,1  ,1/2,1/2,1  ,1  ,0  ,2  ],
  "ground":   [1  ,2  ,1  ,2  ,1/2,1  ,1  ,2  ,1  ,0  ,1  ,1/2,2  ,1  ,1  ,1  ,2  ,1  ],
  "flying":   [1  ,1  ,1  ,1/2,2  ,1  ,2  ,1  ,1  ,1  ,1  ,2  ,1/2,1  ,1  ,1  ,1/2,1  ],
  "water":    [1  ,1  ,1  ,1  ,1  ,1  ,2  ,2  ,1  ,1  ,1/2,1  ,1  ,1  ,1  ,0  ,1/2,1  ],
  "electric": [1  ,1/2,1  ,1  ,2  ,1  ,1/2,1/2,1  ,1/2,2  ,1  ,1  ,1/2,1  ,2  ,1/2,1/2],
  "grass":    [1  ,2  ,1  ,1  ,1  ,2  ,1/2,1  ,1/2,2  ,1  ,2  ,1  ,1  ,1  ,1  ,1/2,1  ],
  "ice":      [0  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,2  ,1  ,1  ,2  ,1  ,1/2,1  ,1  ],
  "fighting": [1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,1  ,2  ,1  ,1/2,0  ],
  "poison":   [1  ,1  ,1  ,1  ,1  ,1  ,1/2,1  ,1  ,1  ,2  ,1  ,1  ,2  ,1  ,1/2,1  ,1/2],
  "ground":   [1  ,1/2,1/2,1/2,1  ,2  ,1  ,1  ,1  ,1  ,1  ,1  ,2  ,1  ,1  ,1  ,1/2,2  ],
  "flying":   [1  ,1/2,1  ,1  ,1  ,1  ,2  ,1/2,1  ,1  ,1  ,1  ,1  ,1  ,2  ,2  ,1/2,1  ]
}
type_number = {
  "normal":   0,
  "fire":     1,
  "water":    2,
  "electric": 3,
  "grass":    4,
  "ice":      5,
  "fighting": 6,
  "poison":   7,
  "ground":   8,
  "flying":   9,
  "water":    10,
  "electric": 11,
  "grass":    12,
  "ice":      13,
  "fighting": 14,
  "poison":   15,
  "ground":   16,
  "flying":   17
}

In [4]:
import requests
import json
import pickle
results = requests.get("https://pokeapi.co/api/v2/pokemon?&limit=1118").json()
pokemons = {}
pokemons_list = []
for pokemon in results["results"]:
  pokemons[pokemon["name"]] = pokemon["url"]
  pokemons_list.append((pokemon["name"], pokemon["url"]))
def get_pokemon_type(args):
  pokemon, url = args
  result = requests.get(url)
  for type in result["types"]:
    
async_multiprocess_loop()


{'bulbasaur': 'https://pokeapi.co/api/v2/pokemon/1/',
 'ivysaur': 'https://pokeapi.co/api/v2/pokemon/2/',
 'venusaur': 'https://pokeapi.co/api/v2/pokemon/3/',
 'charmander': 'https://pokeapi.co/api/v2/pokemon/4/',
 'charmeleon': 'https://pokeapi.co/api/v2/pokemon/5/',
 'charizard': 'https://pokeapi.co/api/v2/pokemon/6/',
 'squirtle': 'https://pokeapi.co/api/v2/pokemon/7/',
 'wartortle': 'https://pokeapi.co/api/v2/pokemon/8/',
 'blastoise': 'https://pokeapi.co/api/v2/pokemon/9/',
 'caterpie': 'https://pokeapi.co/api/v2/pokemon/10/',
 'metapod': 'https://pokeapi.co/api/v2/pokemon/11/',
 'butterfree': 'https://pokeapi.co/api/v2/pokemon/12/',
 'weedle': 'https://pokeapi.co/api/v2/pokemon/13/',
 'kakuna': 'https://pokeapi.co/api/v2/pokemon/14/',
 'beedrill': 'https://pokeapi.co/api/v2/pokemon/15/',
 'pidgey': 'https://pokeapi.co/api/v2/pokemon/16/',
 'pidgeotto': 'https://pokeapi.co/api/v2/pokemon/17/',
 'pidgeot': 'https://pokeapi.co/api/v2/pokemon/18/',
 'rattata': 'https://pokeapi.co/api

In [15]:
import requests
import json
import pickle
moves = requests.get("https://pokeapi.co/api/v2/move?limit=844").json()["results"]
with open("./data/moves.json", "w+") as f:
  json.dump(moves, f)
with open("./data/moves.pickle", "wb") as f:
  pickle.dump(moves, f)

In [5]:
import requests
import json
import pickle
moves = requests.get("https://pokeapi.co/api/v2/type").json()["results"]
with open("./data/types.json", "w+") as f:
  json.dump(moves, f)
with open("./data/types.pickle", "wb") as f:
  pickle.dump(moves, f)

In [1]:
import requests
import pickle
ladder = requests.get("https://pokemonshowdown.com/ladder/gen8randombattle.json").json()
ladder["toplist"][0]

{'userid': 'appelsap',
 'username': 'appelsap',
 'w': 2566,
 'l': 1962,
 't': 0,
 'gxe': 90.3,
 'r': 1920.1336212641,
 'rd': 25,
 'sigma': 0,
 'rptime': '1644138000',
 'rpr': 1920.1336212641,
 'rprd': 25.876418629631,
 'rpsigma': 0,
 'elo': 2454.127109616}

In [1]:
def get_user_replays(user: dict) -> list[dict]:
  userid = user["userid"]
  page = 1
  len_replays = 51
  replays_list = []
  while len_replays == 51 and page < 25:
      replays = requests.get(f"https://replay.pokemonshowdown.com/search.json?user={userid}&page={page}&format=gen8randombattle").json()
      replays_list.extend(replays)
      page += 1
      len_replays = len(replays)
  return userid, replays_list

In [4]:
user_replays = {}
def update(args):
  userid, replays = args
  user_replays[userid] = replays
async_multiprocess_loop(get_user_replays, ladder["toplist"], update)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
import pickle
with open("./data/user_replays.pickle", "wb") as f:
  pickle.dump(user_replays, f)

In [ ]:
total_replays = 0
for replays in user_replays.values():
  total_replays += len(replays)
total_replays

In [ ]:
replays_ids = set()
for user in user_replays.keys():
    for replay in user_replays[user]:
      try:
        replays_ids.add(replay["id"])
      except:
        print(replay)
len(replays_ids)

In [ ]:
import pickle
with open("./data/replay_ids.pickle", "wb") as f:
  pickle.dump(replays_ids, f)

In [5]:
import pickle
with open("./data/replay_ids.pickle", "rb") as f:
    replay_ids_list = list(pickle.load(f))
def get_replay(replay_id: str) -> dict:
  return requests.get(f"https://replay.pokemonshowdown.com/{replay_id}.json").json()

In [5]:
replays = []
def update(replay):
  replays.append(replay)
# async_multiprocess_loop(get_replay, replay_ids_list, update)
# with open("replays.pickle", "wb") as f:
#  pickle.dump(replays, f)
import pickle
with open("./data/replays.pickle", "rb") as f:
  replays = pickle.load(f)

In [18]:
replays[0]

{'id': 'gen8randombattle-1438712292',
 'p1': 'momma umbridge',
 'p2': 'Mario Pratt',
 'format': 'gen8randombattle',
 'log': "|j|☆momma umbridge\n|j|☆Mario Pratt\n|t:|1634704257\n|gametype|singles\n|player|p1|momma umbridge|170|2062\n|player|p2|Mario Pratt|drayden|2049\n|teamsize|p1|6\n|teamsize|p2|6\n|gen|8\n|tier|[Gen 8] Random Battle\n|rated|\n|rule|Species Clause: Limit one of each Pokémon\n|rule|HP Percentage Mod: HP is shown in percentages\n|rule|Sleep Clause Mod: Limit one foe put to sleep\n|\n|t:|1634704257\n|start\n|switch|p1a: Corsola|Corsola, L92, M|269/269\n|switch|p2a: Starmie|Starmie, L84|238/238\n|turn|1\n|inactive|Battle timer is ON: inactive players will automatically lose when time's up. (requested by Mario Pratt)\n|\n|t:|1634704265\n|switch|p2a: Sigilyph|Sigilyph, L83, F|255/255\n|move|p1a: Corsola|Toxic|p2a: Sigilyph\n|-status|p2a: Sigilyph|tox\n|\n|upkeep\n|turn|2\n|\n|t:|1634704277\n|switch|p1a: Togekiss|Togekiss, L80, M|267/267\n|move|p2a: Sigilyph|Energy Ball|p1a

In [6]:
log: list = replays[0]["log"].strip("|").split("\n")
log = log[log.index("|start") + 1:]
log

['|switch|p1a: Corsola|Corsola, L92, M|269/269',
 '|switch|p2a: Starmie|Starmie, L84|238/238',
 '|turn|1',
 "|inactive|Battle timer is ON: inactive players will automatically lose when time's up. (requested by Mario Pratt)",
 '|',
 '|t:|1634704265',
 '|switch|p2a: Sigilyph|Sigilyph, L83, F|255/255',
 '|move|p1a: Corsola|Toxic|p2a: Sigilyph',
 '|-status|p2a: Sigilyph|tox',
 '|',
 '|upkeep',
 '|turn|2',
 '|',
 '|t:|1634704277',
 '|switch|p1a: Togekiss|Togekiss, L80, M|267/267',
 '|move|p2a: Sigilyph|Energy Ball|p1a: Togekiss',
 '|-resisted|p1a: Togekiss',
 '|-damage|p1a: Togekiss|233/267',
 '|',
 '|upkeep',
 '|turn|3',
 '|',
 '|t:|1634704285',
 '|switch|p2a: Xurkitree|Xurkitree, L76|251/251',
 '|move|p1a: Togekiss|Nasty Plot|p1a: Togekiss',
 '|-boost|p1a: Togekiss|spa|2',
 '|',
 '|upkeep',
 '|turn|4',
 '|',
 '|t:|1634704293',
 '|switch|p1a: Dugtrio|Dugtrio, L82, M|192/192',
 '|move|p2a: Xurkitree|Thunderbolt|p1a: Dugtrio',
 '|-immune|p1a: Dugtrio',
 '|',
 '|upkeep',
 '|turn|5',
 '|',
 '|

In [ ]:
from abc import abstractmethod, ABC
class Attribute(ABC):
  def __init__(self, value, hidden=False) -> None:
    self.value = value
    self.hidden = hidden
  def is_hidden(self) -> bool:
    return self.hidden
  def reveal(self):
    self.hidden = True
  def serialize(self) -> object:
    if self.hidden:
      return -1
    else:
      return self.value

class LevelAttribute(Attribute):
  def serialize(self) -> int:
    if self.hidden:
      return -1
    else:
      return self.value / 100

class SexAttribute(Attribute):
  def serialize(self) -> int:
    if self.hidden:
      return -1
    if self.value == "M":
      return 1
    elif self.value == "F":
      return 2
    else:
      return 0

class HPAttribute(Attribute):
  pass

class MoveAttribute(Attribute):
  def serialize(self) -> list:
    raise NotImplementedError()

  def __hash__(self):
    return hash(self.value)

class StatusAttribute(Attribute):
  def serialize(self) -> list:
    raise NotImplementedError()

class PositionAttribute(Attribute):
  pass

class TypeAttribute(Attribute):
  def serialize(self) -> list:
    raise NotImplementedError()

class 

In [21]:
p1_team = {}
p2_team = {}
teams = [p1_team, p2_team]
index_pokemon = [0, 0]
for entry in log:
  items = entry.strip("|").split("|")
  if items[0] == "switch":
    player_index = int(items[1][1:2]) - 1
    pokemon_info = items[2].replace(" ", "").split(",")
    pokemon, level = pokemon_info[0:2]
    level = int(level[1:])
    sex = None
    if len(pokemon_info) == 3:
      sex = pokemon_info[2]
    hp = int(items[3].split("/")[0])
    if pokemon not in teams[player_index]:
      teams[player_index][pokemon] = {"level": level, "sex": sex, "hp": 1.0, "moves": set(), "pos": index_pokemon[player_index], "status": None}
      index_pokemon[player_index] += 1
  elif items[0] == "move":
    player_index = int(items[1][1:2]) - 1
    pokemon = items[1][5:]
    move = items[2]
    teams[player_index][pokemon]["moves"].add(MoveAttribute(move, hidden=player_index==1))
teams

[{'Corsola': {'level': 92,
   'sex': 'M',
   'hp': 1.0,
   'moves': {'Power Gem', 'Toxic'},
   'pos': 0,
   'status': None},
  'Togekiss': {'level': 80,
   'sex': 'M',
   'hp': 1.0,
   'moves': {'Nasty Plot'},
   'pos': 1,
   'status': None},
  'Dugtrio': {'level': 82,
   'sex': 'M',
   'hp': 1.0,
   'moves': {'Earthquake'},
   'pos': 2,
   'status': None},
  'Garchomp': {'level': 74,
   'sex': 'M',
   'hp': 1.0,
   'moves': {'Earthquake'},
   'pos': 3,
   'status': None},
  'Scolipede': {'level': 80,
   'sex': 'M',
   'hp': 1.0,
   'moves': {'Earthquake', 'Megahorn', 'Spikes'},
   'pos': 4,
   'status': None},
  'Kommo-o': {'level': 80,
   'sex': 'F',
   'hp': 1.0,
   'moves': {'Max Knuckle'},
   'pos': 5,
   'status': None}},
 {'Starmie': {'level': 84,
   'sex': None,
   'hp': 1.0,
   'moves': set(),
   'pos': 0,
   'status': None},
  'Sigilyph': {'level': 83,
   'sex': 'F',
   'hp': 1.0,
   'moves': {'Air Slash', 'Energy Ball'},
   'pos': 1,
   'status': None},
  'Xurkitree': {'leve

In [28]:
import copy
class Observation(object):
  def __init__(self):
    self.p1_team = None
    self.p1_active_pokemon = 0
    self.p2_team = None
    self.p2_active_pokemon = 0
  
  def to_array(self) -> list:
    p1_array = [self.p1_active_pokemon, self.p1_team.values()]
    p2_array = [self.p2_active_pokemon, self.p2_team.values()]
    return [p1_array, p2_array]
  
  def copy(self) -> 'Observation':
    new_obs = Observation()
    new_obs.p1_team = copy.deepcopy(self.p1_team)
    new_obs.p2_team = copy.deepcopy(self.p2_team)
    new_obs.p1_active_pokemon = self.p1_active_pokemon
    new_obs.p2_active_pokemon = self.p2_active_pokemon
    return new_obs

In [29]:
turn = 0
observations = [Observation()]
p1 = 0
observations[0].p1_team = teams[p1]
observations[0].p2_team = teams[not p1]
for entry in log:
  items = entry.strip("|").split("|")
  current_obs = observations[turn]
  if items[0] == "switch":
    player_index = int(items[1][1:2]) - 1
    pokemon = items[2].replace(" ", "").split(",")[0]
    if player_index == p1:
      current_obs.p1_active_pokemon = teams[player_index][pokemon]["pos"]
    else:
      current_obs.p2_active_pokemon = teams[player_index][pokemon]["pos"]
  elif items[0] == "-damage":
    player_index = int(items[1][1:2]) - 1
    pokemon = items[1][5:]
    status = None
    if items[2] != "0 fnt":
      hp, total = items[2].split("/")
      if " " in total:
        total, status = total.split(" ")
      hp = float(hp) / float(total)
    else:
      hp = 0
    if player_index == p1:
      current_obs.p1_team[pokemon]["hp"] = hp
      current_obs.p1_team[pokemon]["status"] = status
    else:
      current_obs.p2_team[pokemon]["hp"] = hp
      current_obs.p2_team[pokemon]["status"] = status
  elif items[0] == "turn":
    observations.append(current_obs.copy())
    turn += 1

In [30]:
for observation in observations:
  print(observation.to_array())

[[0, dict_values([{'level': 92, 'sex': 'M', 'hp': 1.0, 'moves': {'Toxic', 'Power Gem'}, 'pos': 0, 'status': None}, {'level': 80, 'sex': 'M', 'hp': 1.0, 'moves': {'Nasty Plot'}, 'pos': 1, 'status': None}, {'level': 82, 'sex': 'M', 'hp': 1.0, 'moves': {'Earthquake'}, 'pos': 2, 'status': None}, {'level': 74, 'sex': 'M', 'hp': 1.0, 'moves': {'Earthquake'}, 'pos': 3, 'status': None}, {'level': 80, 'sex': 'M', 'hp': 1.0, 'moves': {'Megahorn', 'Earthquake', 'Spikes'}, 'pos': 4, 'status': None}, {'level': 80, 'sex': 'F', 'hp': 1.0, 'moves': {'Max Knuckle'}, 'pos': 5, 'status': None}])], [0, dict_values([{'level': 84, 'sex': None, 'hp': 1.0, 'moves': set(), 'pos': 0, 'status': None}, {'level': 83, 'sex': 'F', 'hp': 1.0, 'moves': {'Air Slash', 'Energy Ball'}, 'pos': 1, 'status': None}, {'level': 76, 'sex': None, 'hp': 1.0, 'moves': {'Max Starfall', 'Thunderbolt', 'Max Overgrowth'}, 'pos': 2, 'status': None}, {'level': 69, 'sex': None, 'hp': 1.0, 'moves': {'Oblivion Wing'}, 'pos': 3, 'status': No

In [4]:
import poke_env

len(poke_env.player.env_player.Gen8EnvSinglePlayer._ACTION_SPACE)

22